| Universidad de los Andes<br>Departamento de Ingeniería Eléctrica y Electrónica<br>IELE 4922 - Reinforcement Learning<br><br><p style="font-size:30px">Tutorial : Value y Policy Iteration </p> 	|[<img src="images/uniandes_logo.png" width="250"/>](images/uniandes_logo.png)  	|
|-----------------------------------------------------------------------------------	|---	|

En este notebook se busca implementar los métodos de *Value Iteration* y *Policy Iteration* en un *gridworld*. Con este entorno se puede visualizar de forma gráfica e interactiva el comportamiento de estos métodos. 

# El entorno: *Gridworld*
El entorno consiste en una cuadrícula de 3x4, donde cada celda corresponde a una posible ubicación del agente o estado. El agente en cada celda tiene cuatro acciones posibles de movimiento: norte, sur, este y oeste (N, S, E, O). Sin embargo, estas acciones NO son confiables. Con una probabilidad de 0.8 el agente se moverá en la dirección prevista, y con una probabilidad de 0.2, el agente se moverá en una dirección aleatoria contigua. Es decir si se quiere mover al norte (N), las direcciones contiguas a las que se podrá mover con una probabilidad de 0.2 serán al este (E) o al oeste (O). Adicionalmente, aquellas acciones que lleven al agente fuera de la cuadrícula o a posiciones bloqueadas, no tendrán efecto en la posición del agente.

El estado inicial del agente será en la celda (2,0). Si el agente llega a la celda con el diamante, recibirá una recompensa de +1.0 y el juego (*episodio*) terminará. Si el agente llega a la celda con la bomba, recibirá una recompensa de -1.0 y el episodio terminará. Por ende, los estados (0,3) y (1,3) serán considerados *estados terminales*. El agente recibe recompensa 0.0 por llegar a cualquier otra celda. 

<center>
    <img src="images/gridworld.png" alt="centered image" width=300/>
</center>

**Consideraciones importantes:**

* Para la correcta ejecución del código dado mantenga los nombres de las variables propuestos.

* Debe editar y completar unicamente las celdas que comiencen con la instrucción #EDITABLE

* No se preocupe si la ventana de gridworld no responde en algunos momentos, una vez ejecute los métodos para la evaluación (Todos aquellos deppues de las instrucciones para la **visualización**) esta se actualizará y podrá ver de forma interactiva la actualización de los valores deseados.

* Si quiere volver a visualizar el gridworld, debe volver a ejecutar al menos todas las celdas de esa sección.

In [1]:
# Librerias que contienen la dinámica del entorno gridworld
from types import MethodType
from gridworld import GridWorld, pygame
import copy
import random


pygame 2.2.0 (SDL 2.0.22, Python 3.8.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


Se crea el objeto *gridworld* con las siguientes característica:
* Tamaño de la cuadrícula: 3 filas y 4 columnas
* La celda bloqueda estará en (1,1)
* La celda con la bomba estará en (1,3)
* La celda con el diamante estará en (0,3)

Una vez creado el objeto, en una ventana emergente de pygame se encuentra la visualización del *gridworld*

In [2]:
gw = GridWorld(rows=3, cols=4, walls=[(1,1)], pits=[(1,3)], goals=[(0,3)], live_reward=0.0)
gw

## Espacio de estados
Para acceder a una lista con los estados del gridworld se puede utilizar la propiedad *states* del objeto gridworld, así: `gw.states`

¿Cuántos estados tiene el problema?

In [3]:
# EDITABLE
print("num_states = +"+str(len(gw.states)))
gw.states

num_states = +11


[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3)]

**Resultado esperado:**

```
num_states = 11
```


## Espacio de acciones

Para explorar el espacio de acciones puede hacer uso de la función `gw.get_allowed_actions(state)`, que recibe como parámetro el estado. Este estado se define como una tupla (*x,y*), con la posición donde se encuentra el agente.


¿Cuál es el espacio de acciones en el estado inicial?

In [4]:
# EDITABLE
print("Las acciones posibles en estado inicial (2,0): ")
print("allowed_actions = "+str(gw.get_allowed_actions((2,0))) )


Las acciones posibles en estado inicial (2,0): 
allowed_actions = ['N', 'S', 'E', 'W']


**Resultado esperado:**
```
allowed_actions = ['N', 'S', 'E', 'W']
```

No obstante, se sabe que las acciones no son confiables ya que tienen un ruido inherente. Esto es, con una probabilidad de 0.8 el agente se mueve en la dirección prevista, y con una probabilidad de 0.2, el agente se mueve en una dirección aleatoria hacia los lados.

Por ejemplo, si la acción deseada es 'N', con probabilidad de 0.8 se moverá al norte. Con probabilidad de 0.1 al este y con probabilidad de 0.1 al oeste.

Verifique esto para la acción 'E'. Utilice los atributos `gw.real_actions[action]` y `gw.action_probabilitites`

In [5]:
# EDITABLE
gw.real_actions['E']
#gw.action_probabilities(0,1)

['E', 'N', 'S']

**Resultado esperado:**
```
['E', 'N', 'S']
[0.8, 0.1, 0.1]
```

## Aplicación de la acción
Una vez se conoce el estado actual y la acción que va a ejecutar el agente, esta se puede aplicar al entorno. En respuesta, el agente percibe el nuevo estado del entorno, así como una señal de recompensa que indica que tan buena fue se acción. Igualmente, se recibe una señal de *done* que indica si se alcanzó un estado terminal. En este caso, el episodio finaliza.

<center>
    <img src="images/rl_bloques.PNG" alt="centered image" width=350/>
</center>

La recompensa del agente del gridworld se define como:
\begin{equation*}
R(s,a,s') = \begin{cases}
1 &\text{si $s=(0,3)$}\\
-1 &\text{si $s=(1,3)$}\\
0 &\text{d.l.c}
\end{cases}
\end{equation*}


El método `gw.step(state, action, random)` permite aplicar una acción al gridworld. Este método retorna el nuevo estado, la recompensa, la accion ejecutada y la bandera de terminado (done), todo en ese orden. 

¿Qué sucede si el agente está en el estado (2,1) y realiza la acción 'N' ? Imprima los resultados con el parámetro `random = False`.

In [6]:
# EDITABLE
print("Avanzar step 't' en estado (2,1) aplicar acción N con random=False")
new_state, reward, _, done  = gw.step((2,1), 'N', random=False)
print("new_state = "+str(new_state) )
print("reward = "+str(reward) )
print("action = "+str(_) )
print("done = "+str(done ) )

Avanzar step 't' en estado (2,1) aplicar acción N con random=False
new_state = (2, 1)
reward = 0.0
action = N
done = False


**Resultado esperado**
```
new_state = (2, 1)
reward = 0.0
action = N
done = False
```

Note que el agente, al tratar de moverse en la dirección bloqueada, se queda en su misma posición.

Ahora bien, repita esto con el parámetro `random = True`. En este caso, dada la aleatoriedad del entorno, puede que el resultado obtenido no sea igual al anterior. No obstante, la mayoría de las veces (el 80% para ser exacto) sí lo será. Sientase en la libertad de ejecutar varias veces la celda.

In [7]:
# EDITABLE
print("Aplicar step 't' en estado (2,1) aplicar acción N con random=True")
gw.step((2,1), 'N', random=True)

Aplicar step 't' en estado (2,1) aplicar acción N con random=True


((2, 1), 0.0, 'N', False)

Finalmente, se reinicia el entorno.

In [8]:
print("Reiniciar el entorno")
pygame.quit
gw = None

Reiniciar el entorno


Cierre la ventana emergente y reinicie el kernel de ejecución.

## Función de Valor, $V$

La política $\pi$ indica, para un entorno determinístico, cuál acción debe ejecutarse en cada estado, con el objetivo de que el agente reciba la mayor utilidad. 

Siguiendo una política $\pi$,  el valor de un estado $V(s)$ en el tiempo $t$ formalmente corresponde a **la suma descontada de las recompensas recibidas, si el agente tiene como estado inicial $s$ y se comporta óptimamente en adelante**. En palabras simples, indica cuánta utilidad  recibiría el agente si comenzara en $s$ y se comportara bien de ahí en adelante, 

$$V_{t}(s)=\sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma V_{t-1}(s'))$$

Donde $\gamma$ es un factor conocido como *tasa de descuento*. Permite balancear la preferencia de tener recompensas a corto o largo plazo.

Dada un política $\pi(s)$ es posible encontrar el valor de cada estado por medio de programación dinámica, así ($\gamma=0.9$):
<center>
    <img src="images/fvalor.png" alt="centered image" width=650/>
</center>

Se puede encontrar el valor de cada estado del gridworld a partir de una política fija. Suponga que tenemos la siguiente política determinística:

<center>
    <img src="images/gridworld2.png" alt="centered image" width=250/>
</center>

In [6]:
from types import MethodType
from gridworld import GridWorld, pygame
import copy

gw = GridWorld(3, 4, [(1,1)], [(1,3)], [(0,3)], 0.0)
print("El nuevo MDP: "+str(gw) )

El nuevo MDP: <gridworld.GridWorld object at 0x0000027667986610>


In [10]:
gw.states


[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3)]

Defina la política deterministica propuesta en la figura:

In [12]:
# EDITABLE

state = gw.states
action = ['S','E','E','E','S','N','E','E','E','N','W']
# --------------------------------------------

for i, s in enumerate(state):
    gw.policy[s] = action[i] 
    
print(gw.policy)

{(0, 0): 'S', (0, 1): 'E', (0, 2): 'E', (0, 3): 'E', (1, 0): 'S', (1, 2): 'N', (1, 3): 'E', (2, 0): 'E', (2, 1): 'E', (2, 2): 'N', (2, 3): 'W'}


**Resultado esperado**
```
{(0, 0): 'S', (0, 1): 'E', (0, 2): 'E', (0, 3): 'E', (1, 0): 'S', (1, 2): 'N', (1, 3): 'E', (2, 0): 'E', (2, 1): 'E', (2, 2): 'N', (2, 3): 'W'}
```

Ahora se implementa la función de actualización para estimar los valores de los estados, según la política $\pi(s)$, definida anteirormente. 

**Tenga en cuenta que:**
* Se va a asumir que las acciones dadas por la política son determinísticas. Esto es, se aplican al entorno con probabilidad 1
* La regla de actualización para estados terminales es: $V_{t}(s)=\sum_{s'} P(s'|s,\pi(s))R(s,a,s')$
* La regla de actualización para estados no terminales es: $V_{t}(s)=\sum_{s'} P(s'|s,\pi(s))(R(s,a,s')+\gamma V_{t-1}(s'))$
* Para obtener el valor de un estado en t-1 usamos `v(s)=gw.state_values[state] `

In [3]:
# función de valor de estados: actualizar la función de valor de estados v_pi
# parametros: gw=MDP, gamma=tasa de descuento

def update_values(gw, gamma):
    # vectores para los valores de estado
    
    # valores de estado, para cada estado inicializar en 0 
    value = dict.fromkeys(gw.states , 0.0)
    # nuevos valores de estado, para cada estado inicializar en 0
    new_values = dict.fromkeys(gw.states , 0.0)
    
    
    # iterar sobre los estados posibles del MDP
    for state in gw.states: 
        # Tomar acción de politica dada para el estado
        action = gw.policy[state]
        
        # Dar un paso en par estado-accion (random=False deterministico)
        # retorna: estado siguiente, recompensa, accion
        new_state, reward, _, done = gw.step(state, action, random=False)

        # Actualizar valores de estado
        if(done): # llega a estado terminal
            # actualización para un estado terminal
            new_values[state] = reward # valor de estado para estado terminal es recompensa
        else:
            # actualización para un estado no terminal
            new_values[state] = reward + gamma*gw.state_values[new_state] #borrar
            # valor de estado para estado no terminal es recompensa + descuento*valor de estado[s_t+1]
            
    # Copiar valores
    value = copy.deepcopy(new_values) 
    gw.state_values = copy.deepcopy(new_values) 
    

Para poner a prueba la función se va a escoger una tasa de descuento de 0.9 y a realizar la estimación de los valores durante un horizonte de 15 iteraciones.

In [14]:
gamma = 0.9
H = 15

**Visualización:**
1. Ejecute la siguiente celda y haga *clic* sobre la ventana del gridworld. 
2. Podra ver como se actualizan los valores de cada estado presionando la tecla **Espacio**. 
3. Una vez completetadas las 10 iteraciones, puede cerrar el gridworld presionando la tecla **Esc**.
4. Si tienes un error en la función 'update_values', despues de corregirlo es necesario que vuelvas a correr las 4 celdas anteriores para poder volver a lanzar la interfaz del gridworld

In [15]:
gw.update_values = MethodType(update_values, gw)
gw.solve_dynamic_programming(gamma=gamma, horizon=H)

Al cabo de 10 iteraciones, el valor de los estados, siguiendo la política $\pi(s)$ son:

In [16]:
for state in gw.states:
    print(f'V{state} = {gw.state_values[state]:.3f}')

V(0, 0) = 4.783
V(0, 1) = 8.100
V(0, 2) = 9.000
V(0, 3) = 10.000
V(1, 0) = 5.314
V(1, 2) = 8.100
V(1, 3) = -1.000
V(2, 0) = 5.905
V(2, 1) = 6.561
V(2, 2) = 7.290
V(2, 3) = 6.561


Nota que los estados cercanos a (0,3) tienen valores cercanos y +1. Esto se debe a que el estado terminal (0,3) tiene un valor igual a la recompensa recibida en este estado y se propaga a los demas estados, de acuerdo a la política seguida. Note que los estados mas lejanos a (0,3) como por ejemplo (0,0), (1,0) y (2,0) tienes valores bajos pero siguen siendo positivos. Esto es porque la política define una secuencia de acciones desde cualquiera de estos tres estados al estado terminal (0,3). Sin embargo, la recompensa recibida presenta un mayor descuento, pues se deben realizar más pasos para llegar hasta (0,3).

* Desde (0,0) hasta (0,3) --> 7 pasos --> $v(s)=\gamma^7*R=0.4783$
* Desde (1,0) hasta (0,3) --> 6 pasos --> $v(s)=\gamma^6*R=0.5314$
* Desde (2,0) hasta (0,3) --> 5 pasos --> $v(s)=\gamma^5*R=0.5905$

 Ahora sí, es momento de implementar los métodos que permiten obtener una política para maximizar las recompensas.

## Value iteration $V^{*}$

Este método permite aproximar los valores óptimos de los estados, de tal manera que se pueda encontrar, al final, una política $\pi^{*}(s)$ que permita maximizar la recompensa recibida por el agente a lo largo del horizonte.

$$V^{*}(s)=\max_{\pi} \mathbb{E}\left[ \sum_{t=0}^{H} \gamma^{t} R(s_t,a_t,s_{t+1}) | \pi, s_0=s \right]$$

Cuando el horizonte es $H=0$, el valor de todos los estados es igual a su valor de inicialización, generalmente cero.

In [7]:
from types import MethodType
from gridworld import GridWorld, pygame
import copy
from colorama import Fore

gw = GridWorld(3, 4, [(1,1)], [(1,3)], [(0,3)], 0.0)


print("El MDP: "+str(gw))

El MDP: <gridworld.GridWorld object at 0x00000276679867C0>


In [8]:
# Inicializar valores de los estados
gw.state_values = gw.init_values()
print("Valores de estados iniciales: "+str(gw.state_values) )

Valores de estados iniciales: {(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (1, 0): 0.0, (1, 2): 0.0, (1, 3): 0.0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0, (2, 3): 0.0}


Verifique los valores de los siguientes estados: (2,0), (0,3), (1,3)

In [9]:
# EDITABLE
print("Valor de estado 2,0: "+str(gw.state_values[2,0]) )
print("Valor de estado 0,3: "+str(gw.state_values[0,3]) )
print("Valor de estado 1,3: "+str(gw.state_values[1,3]) )

Valor de estado 2,0: 0.0
Valor de estado 0,3: 0.0
Valor de estado 1,3: 0.0


**Resultado esperado**
```
V(2,0) = 0.0
V(0,3) = 0.0
V(1,3) = 0.0
```

Cuando el horizonte es $H=1$, el valor del estado $s$ corresponde la recompensa recibida para todas las transiciones a $s'$ posibles + el valor del nuevo estado $s'$:
$$V_{1}^{*}(s)=\max_a \sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma V_{0}^{*}(s'))$$

Pero si en $H=1$ el agente está en un estado terminal, por ejemplo (0,3), el valor de $V_1(0,3)$ se actualizaría así:
$$V_{1}^{*}(0,3)=\max_a \sum_{s'} P(s'|s,a)(R(s,a,s')$$
En este caso se omite el término $V_{0}^{*}(s')$ ya que esta transición no existe al finalizarce el episodio. Por esta razón, "el valor de los estados terminales es igual a la recompensa recibida en ellos"




In [10]:
# Se ajusta el valor de los estados terminales

# definir valor de estados terminales
gw.state_values[(0,3)] = 1.0
gw.state_values[(1,3)] = -1.0  
gw.state_values

{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 1.0,
 (1, 0): 0.0,
 (1, 2): 0.0,
 (1, 3): -1.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 0.0,
 (2, 3): 0.0}

Si el horizonte es $H=2$, para estados no terminales:
$$V_{2}^{*}(s)=\max_a \sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s'))$$
Por ejemplo, si el agente está en (0,2) y se tiene un descuento $\gamma=0.90$:

* Si pretende ejecutar 'E':
    * Si efectivamente se ejecuta 'E':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.8(0.0+\gamma V_1(0,3))$
    * Si, por ruido, se ejecuta 'N':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,2))$
    * Si, por ruido, se ejecuta 'S':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(1,2))$
    * Valor = $0.8(0.9*1.0)+0.1(0.0)+0.1*(0.0)=0.72$
    
* Si pretende ejecutar 'W':
    * Si efectivamente se ejecuta 'W':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.8(0.0+\gamma V_1(0,1))$
    * Si, por ruido, se ejecuta 'N':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,2))$
    * Si, por ruido, se ejecuta 'S':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(1,2))$
    * Valor = $0.8(0.0)+0.1(0.0)+0.1*(0.0)=0.0$

* Si pretende ejecutar 'N':
    * Si efectivamente se ejecuta 'N':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.8(0.0+\gamma V_1(0,2))$
    * Si, por ruido, se ejecuta 'E':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,3))$
    * Si, por ruido, se ejecuta 'W':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,1))$
    * Valor = $0.8(0.0)+0.1(0.9*1.0)+0.1*(0.0)=0.09$
    
* Si pretende ejecutar 'S':
    * Si efectivamente se ejecuta 'S':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.8(0.0+\gamma V_1(1,2))$
    * Si, por ruido, se ejecuta 'E':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,3))$
    * Si, por ruido, se ejecuta 'W':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,1))$
    * Valor = $0.8(0.0)+0.1(0.9*1.0)+0.1*(0.0)=0.09$
    
Por ende, la acción que maximiza el valor del estado (0,2) es 'E' y $V_{2}^{*}(0,2)=0.72$

### Algoritmo
>Inicializar el valor de todos los estados como $V_{0}^{*}(s)=0$<br>
>Para $k=1, \cdots, H$:<br>
>> Para todos los estados $s$:<br>
>>> $\displaystyle V_{k}^{*}(s)=\max_a \sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma V_{k-1}^{*}(s'))$<br>
>>> $\displaystyle \pi_{k}^{*}(s)=arg\max_a \sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma V_{k-1}^{*}(s'))$<br>

        

In [3]:
# EDITABLE

# value_iteration: encontrar valores de estado óptimos para cada estado y política óptima para cada estado 
# params: gw=MDP gridworld, gamma=descuento

def value_iteration(gw,gamma):   
    
    # valores de estado, para cada estado inicializar en 0 
    value = dict.fromkeys(gw.states , 0.0)
    
    # nuevos valores de estado, para cada estado inicializar en 0
    new_values = dict.fromkeys(gw.states , 0.0)    
    
    
    # politica aleatoria 
    # inicializar politica aleatoria 
    gw.policy = {}
    keys = gw.states # keys del diccionario son los estados posibles del MDP
    
    # iterar sobre estados posibles del MDP
    for state in keys:    
        # inicializar valores de política aleatoria escogiendo acción aleatoria en estado 
        gw.policy[state] = random.choice(gw.get_allowed_actions(state) )
    print("Política aleatoria incial: "+str(gw.policy) )    
    
    # iterar sobre estados
    for state in gw.states:          
        
        # Dar un paso en par estado-accion con accion dada por la política aleatoria DETERMINISTICO
        new_state, reward, _, done = gw.step(state, gw.policy[(state)], random=False)
        
        if(done==True): # llega a estado terminal
            # actualización para un estado terminal        
            new_values[state] = reward # valor de estado para estado terminal es recompensa
            print("Estado terminal: "+str(state) )
            print("Recompensa estado terminal: "+str(reward) )
        else: # llega a estado no terminal
            # actualización para un estado no terminal
            
            accion_opt = -1 # acción óptima en estado (política)
            val_estado_opt = -100 # valor óptimo en estado 
            print("Estado actual: "+str(state) )
            
            
            # iterar sobre acciones permitidas en estado state (accion - E)
            for action in gw.get_allowed_actions(state):
                
                suma_actions = 0 # suma de valores de par estado-accion real que puede tomar (en un estado)
                # iterar sobre Las posibles acciones (reales) que puede tomar el agente dada una acción
                
                for real_action in  gw.real_actions[action]:
                    # si acción real es igual a la acción que toma: probabilidad 0.8
                    if real_action==action: 
                        new_state, reward, _, done = gw.step(state, real_action, random=False)
                        suma_actions+=gw.action_probabilities[0]*(reward+gamma*gw.state_values[new_state]) #probabilidad 0.8
                        
                    # si acción real es diferente a la acción que toma:  probabilidad 0.1
                    else: 
                        new_state, reward, _, done = gw.step(state, real_action, random=False)
                        suma_actions+=gw.action_probabilities[1]*(reward+gamma*gw.state_values[new_state]) #probabilidad 0.1
                
                print("Suma valor de acciones para acción "+str(action)+' : '+str( suma_actions) )
                if suma_actions>val_estado_opt:
                    val_estado_opt = suma_actions
                    accion_opt=action
        
            # actualizar valor de estado con el valor óptimo de la acción
            new_values[state]= val_estado_opt
            # actualizar política con acción óptima para el estado
            gw.policy[state] = accion_opt
            
            
            print("Valor de estado óptimo: "+str(new_values[state]) )
            print("Acción óptima en estado: "+str(gw.policy[state]) )
            print("")
            
    # Copiar valores
    value = copy.deepcopy(new_values) 
    gw.state_values = copy.deepcopy(new_values)         
    print("Valores de estado óptimos: "+str(new_values) )
    print("Política óptima: "+str (gw.policy))
        
value_iteration(gw, 0.9)

#gw.get_allowed_actions(state): Permite conocer el espacio de acciones dado un estado.
#gw.real_actions[action]: Las posibles acciones (reales) que puede tomar el agente dada una acción.
#gw.action_probabilities[action]: La probabilidad de que el agente tomé las acciones reales.
#gw.step(state, action, random): Permite ejecutar un paso en el gridworld (MDP). Con este método se puede conocer el estado siguiente, la recompensa, la accion tomada y la bandera de un estado terminal. Nota: si el parámetro random es Falso, las acciones son determinisiticas (con probabilidad 1, toma la acción dada).
#gw.state_values[state]: Los valores del estado state.
#gw.policy[state]: La acción dada por la política para el estado state

Política aleatoria incial: {(0, 0): 'W', (0, 1): 'W', (0, 2): 'N', (0, 3): 'E', (1, 0): 'W', (1, 2): 'W', (1, 3): 'E', (2, 0): 'W', (2, 1): 'E', (2, 2): 'S', (2, 3): 'S'}
Estado actual: (0, 0)
Suma valor de acciones para acción N : 0.0
Suma valor de acciones para acción S : 0.0
Suma valor de acciones para acción E : 0.0
Suma valor de acciones para acción W : 0.0
Valor de estado óptimo: 0.0
Acción óptima en estado: N

Estado actual: (0, 1)
Suma valor de acciones para acción N : 0.0
Suma valor de acciones para acción S : 0.0
Suma valor de acciones para acción E : 0.0
Suma valor de acciones para acción W : 0.0
Valor de estado óptimo: 0.0
Acción óptima en estado: N

Estado actual: (0, 2)
Suma valor de acciones para acción N : 0.0
Suma valor de acciones para acción S : 0.0
Suma valor de acciones para acción E : 0.0
Suma valor de acciones para acción W : 0.0
Valor de estado óptimo: 0.0
Acción óptima en estado: N

Estado terminal: (0, 3)
Recompensa estado terminal: 1.0
Estado actual: (1, 0)
S

In [4]:
# EDITABLE

# value_iteration: encontrar valores de estado óptimos para cada estado y política óptima para cada estado 
# params: gw=MDP gridworld, gamma=descuento

def value_iteration_1(gw,gamma):   
    
    # valores de estado, para cada estado inicializar en 0 
    value = dict.fromkeys(gw.states , 0.0)
    
    # nuevos valores de estado, para cada estado inicializar en 0
    new_values = dict.fromkeys(gw.states , 0.0)    
    
    
    # politica aleatoria 
    # inicializar politica aleatoria 
    gw.policy = {}
    keys = gw.states # keys del diccionario son los estados posibles del MDP
    
    # iterar sobre estados posibles del MDP
    for state in keys:    
        # inicializar valores de política aleatoria escogiendo acción aleatoria en estado 
        gw.policy[state] = random.choice(gw.get_allowed_actions(state) )
    print("Política aleatoria incial: "+str(gw.policy) )    
    
    # iterar sobre estados
    for state in gw.states:          
        
        # si estado es terminal
        if(state in gw.goals or state in gw.pits): # llega a estado terminal
            # actualización para un estado terminal        
            new_values[state] = 0 # valor de estado para estado terminal es 0 v(st)=0
            print("Estado terminal: "+str(state) )
            print("Recompensa estado terminal: "+str(reward) )
        # si estado es no terminal
        else: 
                    
            accion_opt = -1 # acción óptima en estado (política)
            val_estado_opt = -100 # valor óptimo en estado 
            print("Estado actual: "+str(state) )
            
            # iterar sobre acciones permitidas en estado state 
            for action in gw.get_allowed_actions(state): #['N','S','E','W']
                
                suma_actions = 0 # suma de valores de par estado-accion real que puede tomar (en un estado)
                # iterar sobre Las posibles acciones (reales) que puede tomar el agente dada una acción
                
                for real_action in  gw.real_actions2[action]: # 
                    # si acción real es igual a la acción que toma: probabilidad 0.8
                    if real_action==action: 
                        new_state, reward, _, done = gw.step(state, real_action, random=False)
                        suma_actions+=gw.action_probabilities2[0]*(reward+gamma*gw.state_values[new_state]) #probabilidad 0.8
                        
                    # si acción real es diferente a la acción que toma:  probabilidad 0.2
                    else: 
                        new_state, reward, _, done = gw.step(state, real_action, random=False)
                        suma_actions+=gw.action_probabilities2[1]*(reward+gamma*gw.state_values[new_state]) #probabilidad 0.2
                        
                print("Suma valor de acciones para acción "+str(action)+' : '+str( suma_actions) )
                if suma_actions>val_estado_opt:
                    val_estado_opt = suma_actions
                    accion_opt=action
        
            # actualizar valor de estado con el valor óptimo de la acción
            new_values[state]= val_estado_opt
            # actualizar política con acción óptima para el estado
            gw.policy[state] = accion_opt
            
            
            print("Valor de estado óptimo: "+str(new_values[state]) )
            print("Acción óptima en estado: "+str(gw.policy[state]) )
            print("")
            
    # Copiar valores
    value = copy.deepcopy(new_values) 
    gw.state_values = copy.deepcopy(new_values)         
    print("Valores de estado óptimos: "+str(new_values) )
    print("Política óptima: "+str (gw.policy))
        
value_iteration_1(gw, 0.9)

Política aleatoria incial: {(0, 0): 'E', (0, 1): 'S', (0, 2): 'N', (0, 3): 'E', (1, 0): 'S', (1, 2): 'E', (1, 3): 'N', (2, 0): 'N', (2, 1): 'E', (2, 2): 'W', (2, 3): 'N'}
Estado actual: (0, 0)
Suma valor de acciones para acción N : 0.0
Suma valor de acciones para acción S : 0.0


AssertionError: 

: 

## Métodos de clase Gridworld (MDP) 

In [10]:
print("acciones dado un estado"+str(gw.get_allowed_actions((0,0))))

acciones dado un estado['N', 'S', 'E', 'W']


In [11]:
print("acciones que puede tomar el agente dada una acción"+str(gw.real_actions['N']) )

acciones que puede tomar el agente dada una acción['N', 'E', 'W']


In [12]:
print("Probabilidad de acción 1 (real): "+str(gw.action_probabilities[0]))
print("Probabilidad de acción 2 (ruido1): "+str(gw.action_probabilities[1]))
print("Probabilidad de acción 3 (ruido2): "+str(gw.action_probabilities[2]))

Probabilidad de acción 1 (real): 0.8
Probabilidad de acción 2 (ruido1): 0.1
Probabilidad de acción 3 (ruido2): 0.1


In [13]:
state = (0,2) # estado anterior
action = 'E' # acción estado anterior

# Dar un paso en par estado-accion (random=[False deterministico/True estocástico] )
new_state, reward, _, done = gw.step(state, action, random=False)
print("estado siguiente: "+str(new_state) )
print("recompensa estado siguiente: "+str(reward) )
print("accion anterior: "+str(_))
print("bandera estado siguiente: "+str(done) )

estado siguiente: (0, 3)
recompensa estado siguiente: 0.0
accion anterior: E
bandera estado siguiente: False


In [16]:
print("Valores de estado actuales v_pi(0,2): ")
gw.state_values[(0,3)]

Valores de estado actuales v_pi(0,2): 


10.0

In [17]:
print("Acción dada por la política para el estado (0,2): ")
gw.policy[(0,2)]

Acción dada por la política para el estado (0,2): 


'N'

Implementemos el algoritmo de value iteration. Las siguientes funciones o atributos pueden ser útiles:
* `gw.get_allowed_actions(state)`: Permite conocer el espacio de acciones dado un estado.
* `gw.real_actions[action]`: Las posibles acciones (reales) que puede tomar el agente dada una acción.
* `gw.action_probabilities[action]`: La probabilidad de que el agente tomé las acciones reales.
* `gw.step(state, action, random)`: Permite ejecutar un paso en el gridworld (MDP). Con este método se puede conocer el estado siguiente, la recompensa, la accion tomada y la bandera de un estado terminal. Nota: si el parámetro random es Falso, las acciones son determinisiticas (con probabilidad 1, toma la acción dada).
* `gw.state_values[state]`: Los valores del estado *state*.
* `gw.policy[state]`: La acción dada por la política para el estado *state*.

**Tenga en cuenta:** Si tiene un error en la función 'value_iteration', despues de corregirlo es necesario que vuelva a ejecutar la celda anterior para tener en cuenta el cambio

Verifique con la siguiente celda que el valor del estado (0,2) es el correcto.

In [13]:
# EDITABLE
value_iteration(gw, 0.9)

Política aleatoria incial: {(0, 0): 'N', (0, 1): 'W', (0, 2): 'E', (0, 3): 'S', (1, 0): 'E', (1, 2): 'N', (1, 3): 'S', (2, 0): 'S', (2, 1): 'S', (2, 2): 'S', (2, 3): 'W'}
Estado actual: (0, 0)
Suma valor de acciones para acción N : 0.0
Suma valor de acciones para acción S : 0.0
Suma valor de acciones para acción E : 0.0
Suma valor de acciones para acción W : 0.0
Valor de estado óptimo: 0.0
Acción óptima en estado: N

Estado actual: (0, 1)
Suma valor de acciones para acción N : 0.06480000000000001
Suma valor de acciones para acción S : 0.06480000000000001
Suma valor de acciones para acción E : 0.5184000000000001
Suma valor de acciones para acción W : 0.0
Valor de estado óptimo: 0.5184000000000001
Acción óptima en estado: E

Estado actual: (0, 2)
Suma valor de acciones para acción N : 0.6084
Suma valor de acciones para acción S : 0.09000000000000001
Suma valor de acciones para acción E : 0.7848
Suma valor de acciones para acción W : 0.06480000000000001
Valor de estado óptimo: 0.7848
Acci

In [12]:
# EDITABLE
value_iteration2(gw, 0.9)

NameError: name 'value_iteration2' is not defined

**Resultado esperado\*:**
```
Para H=2, V(0, 2) = 0.7200000000000001
Para H=2, acción que maximiza el valor = E
```

\* Recuerde reiniciar los valores y ejecutar una única vez el entorno si desea tener un horizonte H=2.

Ahora resuelva el gridworld por *value iteration* utilizando la función que usted implementó:
* Incialice los estados iniciales
* Establezca un $\gamma=0.90$, un horizonte de 15 iteraciones y el estado inicial en (2,0)

In [24]:
# EDITABLE
# Se define una política inicial cualquiera
#gw.policy = dict.fromkeys(gw.states, 'N')

from types import MethodType
from gridworld import GridWorld, pygame
import copy

# crear MDP=GridWorld
# params: 3 filas, 4 columnas,  (1,1)=celda bloqueada, (1,3)=bomba, (0,3)=diamante, 0.0=valor de estados inicial
gw = GridWorld(3, 4, [(1,1)], [(1,3)], [(0,3)], 0.0)
print("El nuevo MDP: "+str(gw) )

# inicializar valores de estado

# definir valor de estados terminales
gw.state_values[(0,3)] = 1.0
gw.state_values[(1,3)] = -1.0  
init_state= gw.state_values

gamma=0.9 # tasa descuento
H=15 # horizonte de tiempo

init_state

El nuevo MDP: <gridworld.GridWorld object at 0x000001A53B12B190>


{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 1.0,
 (1, 0): 0.0,
 (1, 2): 0.0,
 (1, 3): -1.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 0.0,
 (2, 3): 0.0}

**Visualización:**
1. Ejecute la siguiente celda y haga *clic* sobre la ventana del gridworld. 
2. Podra ver como se actualizan los valores de cada estado presionando la tecla **Espacio**. 
3. Una vez completes las 30 iteraciones, presiona la tecla **Enter** para ver la política
4. Puede cerrar el gridworld presionando la tecla **Esc**.

In [18]:
gamma=0.9
H=30
init_state=(2,0)
gw.value_iteration = MethodType(value_iteration, gw)
gw.solve_value_iteration(gamma=gamma, horizon=H, init_state=init_state )

Política aleatoria incial: {(0, 0): 'N', (0, 1): 'N', (0, 2): 'E', (0, 3): 'W', (1, 0): 'N', (1, 2): 'E', (1, 3): 'E', (2, 0): 'N', (2, 1): 'E', (2, 2): 'S', (2, 3): 'E'}
Estado actual: (0, 0)
Suma valor de acciones para acción N : 0.04665600000000001
Suma valor de acciones para acción S : 0.04665600000000001
Suma valor de acciones para acción E : 0.3732480000000001
Suma valor de acciones para acción W : 0.0
Valor de estado óptimo: 0.3732480000000001
Acción óptima en estado: E

Estado actual: (0, 1)
Suma valor de acciones para acción N : 0.4438800000000001
Suma valor de acciones para acción S : 0.4438800000000001
Suma valor de acciones para acción E : 0.6583680000000002
Suma valor de acciones para acción W : 0.09331200000000002
Valor de estado óptimo: 0.6583680000000002
Acción óptima en estado: E

Estado actual: (0, 2)
Suma valor de acciones para acción N : 0.7017120000000001
Suma valor de acciones para acción S : 0.4451040000000001
Suma valor de acciones para acción E : 0.829188000000

AssertionError: 

: 

In [14]:
gw.value_iteration = MethodType(value_iteration2, gw)
gw.solve_value_iteration(gamma=gamma, horizon=H, init_state=init_state )

NameError: name 'value_iteration2' is not defined

Para un horizonte de 15, los valores de los estados son: 

In [31]:
# EDITABLE
print("Valores de estado finales: ")
print(gw.state_values)

Valores de estado finales: 
{(0, 0): 6.522126605634782, (0, 1): 7.527653273916826, (0, 2): 8.573167435949768, (0, 3): 10.0, (1, 0): 5.726220838932412, (1, 2): 6.684260885746523, (1, 3): -1.0, (2, 0): 5.020195701002069, (2, 1): 4.962532283609661, (2, 2): 5.65263751543796, (2, 3): 4.373005815555748}


**Resultado esperado:**
```
V(0, 0) = 0.645
V(0, 1) = 0.744
V(0, 2) = 0.848
V(0, 3) = 1.000
V(1, 0) = 0.566
V(1, 2) = 0.572
V(1, 3) = -1.000
V(2, 0) = 0.491
V(2, 1) = 0.431
V(2, 2) = 0.475
V(2, 3) = 0.277
```

La política aprendida es para maximizar la recompensa desde el estado inicial (2,0) es:

In [32]:
# EDITABLE
print("Política aprendida: ")
print(gw.policy)

Política aprendida: 
{(0, 0): 'E', (0, 1): 'E', (0, 2): 'E', (0, 3): 'S', (1, 0): 'N', (1, 2): 'N', (1, 3): 'W', (2, 0): 'N', (2, 1): 'E', (2, 2): 'N', (2, 3): 'W'}


**Resultado esperado:**
```
Acción a tomar en (0, 0): E
Acción a tomar en (0, 1): E
Acción a tomar en (0, 2): E
Acción a tomar en (0, 3): N
Acción a tomar en (1, 0): N
Acción a tomar en (1, 2): N
Acción a tomar en (1, 3): N
Acción a tomar en (2, 0): N
Acción a tomar en (2, 1): W
Acción a tomar en (2, 2): N
Acción a tomar en (2, 3): W
```

## Policy Iteration

Es otro método que busca aproximar los valores óptimos de cada estado basados en la política actual $\pi(s)$. La politica puede mejorarse con el tiempo, de acuerdo a los valores Q de cada par estado-acción.

### Valores Q
$Q^{*}(s,a)$ es el valor esperado de la utilidad, si el agente comenzara en el estado $s$, tomando la acción $a$ y comportandose óptimamente en adelante.

Los valores Q pueden aproximarse así:
$$Q_{k+1}^{*}(s,a)\leftarrow \sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma \max_{a'}Q_{k}^{*}(s',a'))$$

In [56]:
from types import MethodType
from gridworld import GridWorld, pygame
import copy
from colorama import Fore

gw = GridWorld(3, 4, [(1,1)], [(1,3)], [(0,3)], 0.0)

Se inicializan los valores de los estados en 0.0, al igual que los valores Q para todo par estado acción. Además, inicializamos una política que por defecto siempre toma el norte.

In [58]:
# Inicializar valores y política
import random
gw.state_values = gw.init_values()
gw.state_q_values = gw.init_qvalues()

# definir valor de estados terminales
gw.state_values[(0,3)] = 1.0
gw.state_values[(1,3)] = -1.0  

gw.policy = dict.fromkeys(gw.states, 'N') # cambiar política
gw.updateGrid()

gw.state_values

{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 1.0,
 (1, 0): 0.0,
 (1, 2): 0.0,
 (1, 3): -1.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 0.0,
 (2, 3): 0.0}

Verifique la cantidad de valores q:

In [59]:
# EDITABLE
gw.state_q_values
#type(gw.state_q_values)

{(0, 0): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0},
 (0, 1): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0},
 (0, 2): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0},
 (0, 3): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0},
 (1, 0): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0},
 (1, 2): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0},
 (1, 3): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0},
 (2, 0): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0},
 (2, 1): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0},
 (2, 2): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0},
 (2, 3): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}}

**Resultado esperado:**

```
num_q_values = 44
```

### Algoritmo

> Inicializar $\pi_{0}(s)$ de forma arbitraria<br>
> Inicializar $V_{0}(s)$ y $Q_{0}(s,a)$ en 0.0 para todos los estados y acciones<br>
> policy_stable = False<br>
> Mientras(not policy_stable):<br>
>> 1. Evaluar la política<br>
>> Para $k=1,\cdots,H$:<br>
>>> Para todos los estados $s$:<br>
>>>> Para todas las acciones $s$ permitidas en $s$:<br>
>>>>> $$Q_{k}(s,a)=\sum_{s'} P(s'|s,\pi(s))(R(s,a,s')+\gamma Q_{k-1}(s',\pi(s')))$$<br>
>>>> Valor del estado $s$ corresponde al valor Q de la acción a ejecutar según la política $\pi_k(s)$ <br>
>>>> $V_{k}(s)=Q_{k}(s,\pi_{k}(s))$ <br>
>
>> 2. Mejorar la política <br>
>> policy_stable = True <br>
>> Para todos los estados $s$:<br>
>>> best_action = $arg\max_{a}Q_k(s,)$<br>
>>> si *best_action* es diferente a lo que dice $\pi_{k}(s)$:<br>
>>>> $\pi_{k}(s) = best\_action$<br>
>>>> policy_stable = False<br>

Implemente la función de *policy evaluation* en donde se actualizan los valores $Q(s,a)$ y $V(s)$ de acuerdo a la política:

Las siguientes funciones pueden ser útiles:

* `gw.get_allowed_actions(state)`: Permite conocer el espacio de acciones.
* `gw.real_actions[action]`: Las posibles acciones (reales) que puede tomar el agente.
* `gw.action_probabilities[action]`: La probabilidad de que el agente tomé las acciones reales.
* `gw.step(state, action, random)`: Permite ejecutar un paso en el gridworld. Con este método se puede conocer el estado siguiente, la recompensa, la accion tomada y la bandera de un estado terminal. Nota: si el parámetro random es Falso, las acciones son determinisiticas (con probabilidad 1, toma la acción dada).
* `gw.state_values[state]`: Los valores del estado *state*.
* `gw.policy[state]`: La acción dada por la política para el estado *state*.

In [15]:
gw.get_allowed_actions((0,0))[0]

'N'

In [91]:
# EDITABLE

# policy_evaluation: encontrar valores de estado a través de la función de valor par estado-acción óptimos y política óptima para cada estado 
# params: gw=MDP gridworld, gamma=descuento

def policy_evaluation(gw,gamma):   
    
    # valores de estado, para cada estado inicializar en 0 
    value = dict.fromkeys(gw.states , 0.0)
    
    # nuevos valores de estado, para cada estado inicializar en 0
    new_values = dict.fromkeys(gw.states , 0.0)    
    
    # valores de par estado-accion, para cada estado inicializar en 0
    state_q_values = gw.init_qvalues()
    # nuevos valores de par estado-accion, para cada estado inicializar en 0
    new_state_q_values = gw.init_qvalues()
    
    # politica aleatoria 
    # inicializar politica aleatoria 
    gw.policy = {}
    keys = gw.states # keys del diccionario son los estados posibles del MDP
    
    # iterar sobre estados posibles del MDP
    for state in keys:    
        # inicializar valores de política aleatoria escogiendo acción aleatoria en estado 
        gw.policy[state] = random.choice(gw.get_allowed_actions(state) )
        
        
    # iterar sobre estados
    for state in gw.states:          
        #print("Estado actual")
        # Dar un paso en par estado-accion con accion dada por la política
        new_state, reward, _, done = gw.step(state, gw.policy[(state)], random=True)
        
        # actualización para un estado terminal
        if(done==True): # llega a estado terminal             
            new_values[state] = reward # valor de estado para estado terminal es recompensa
            
            #gw.state_values[state] = reward
            print("Estado terminal: "+str(state) )
            print("Recompensa estado terminal: "+str(reward) )
            
        else: # llega a estado no terminal
            print("Estado actual: "+str(state) )
            # actualización para un estado no terminal          
            
            # iterar sobre acciones permitidas en estado state
            for action in gw.get_allowed_actions(state):
                
                suma_actions = 0.0 # suma de valores de par estado-accion real que puede tomar (en un estado)
                
                # iterar sobre las posibles acciones (reales) que puede tomar el agente dada una acción
                for real_action in  gw.real_actions[action]:
                    # si acción real es igual a la acción que toma: probabilidad 0.8
                    if real_action==action: 
                        new_state, reward, _, done = gw.step(state, real_action, random=False)
                        suma_actions+=gw.action_probabilities[0]*(reward+gamma*gw.state_q_values[new_state][real_action]) #probabilidad 0.8

                    # si acción real es diferente a la acción que toma:  probabilidad 0.1
                    else: 
                        new_state, reward, _, done = gw.step(state, real_action, random=False)
                        suma_actions+=gw.action_probabilities[1]*(reward+gamma*gw.state_q_values[new_state][real_action]) #probabilidad 0.1
                
                
                print("Q(s,a) en acción: "+str(action) )
                print(suma_actions)
                # la suma de valor de par estado-accion para todas las acciones posibles, dada una acción es el nuevo valor de par estado-accion
                new_state_q_values[state][action] = suma_actions
                
            new_values[state] = new_state_q_values[state][gw.policy[state]]  
            #gw.state_values[state] =  gw.max_val(gw.state_q_values[state])
            
            print("Nuevo valor de estado V(s): "+ str(new_values[state]) )
            print("")
            
    # Copiar valores
    value = copy.deepcopy(new_values) 
    gw.state_values = copy.deepcopy(new_values) 
    
    state_q_values = copy.deepcopy(new_state_q_values) 
    gw.state_q_values = copy.deepcopy(new_state_q_values) 
    policy_improvement(gw)
    
    
policy_evaluation(gw,0.9)      

Estado actual: (0, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 1)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (0, 3)
Recompensa estado terminal: 1.0
Estado actual: (1, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (1, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (1, 3)
Recompensa estado terminal: -1.0
Estado actual: (2, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de

**Resultado esperado:**
```
V(0, 0) = 0.951
V(0, 1) = 0.964
V(0, 2) = 0.977
V(0, 3) = 1.000
V(1, 0) = 0.939
V(1, 2) = 0.890
V(1, 3) = -1.000
V(2, 0) = 0.925
V(2, 1) = 0.913
V(2, 2) = 0.900
V(2, 3) = 0.790
```


In [85]:
# EDITABLE

# policy_evaluation: encontrar valores de par estado-acción óptimos y política óptima para cada estado 
# params: gw=MDP gridworld, gamma=descuento

def policy_evaluation(gw,gamma):   
    
    # valores de estado, para cada estado inicializar en 0 
    value = dict.fromkeys(gw.states , 0.0)
    
    # nuevos valores de estado, para cada estado inicializar en 0
    new_values = dict.fromkeys(gw.states , 0.0)    
    
    # valores de par estado-accion, para cada estado inicializar en 0
    states = [gw.states]
    action_values = {gw.get_allowed_actions((0,0))[0]: 0.0, gw.get_allowed_actions((0,0))[1]: 0.0, gw.get_allowed_actions((0,0))[2]: 0.0, gw.get_allowed_actions((0,0))[3]: 0.0}
    
    state_q_values= {key: action_values[k] for key in states}
    

    
    # politica aleatoria 
    # inicializar politica aleatoria 
    gw.policy = {}
    keys = gw.states # keys del diccionario son los estados posibles del MDP
    
    # iterar sobre estados posibles del MDP
    for state in keys:    
        # inicializar valores de política aleatoria escogiendo acción aleatoria en estado 
        gw.policy[state] = random.choice(gw.get_allowed_actions(state) )
    
    print("Política aleatoria incial: "+str(gw.policy) )    
    
    
    # iterar sobre estados
    for state in gw.states:          
        
        # Dar un paso en par estado-accion con accion dada por la política aleatoria
        new_state, reward, _, done = gw.step(state, gw.policy[(state)], random=True)
        
        if(done==True): # llega a estado terminal
            # actualización para un estado terminal        
            new_values[state] = reward # valor de estado para estado terminal es recompensa
            print("Estado terminal: "+str(state) )
            print("Recompensa estado terminal: "+str(reward) )
        else: # llega a estado no terminal
            # actualización para un estado no terminal
            
            accion_opt = -1 # acción óptima en estado (política)
            val_estado_opt = -100 # valor óptimo en estado 
            print("Estado actual: "+str(state) )
            # iterar sobre acciones permitidas en estado state
            for action in gw.get_allowed_actions(state):
                
                suma_actions = 0 # suma de valores de par estado-accion real que puede tomar (en un estado)
                # iterar sobre Las posibles acciones (reales) que puede tomar el agente dada una acción
                for real_action in  gw.real_actions[action]:
                    # si acción real es igual a la acción que toma: probabilidad 0.8
                    if real_action==action: 
                        new_state, reward, _, done = gw.step(state, real_action, random=False)
                        suma_actions+=gw.action_probabilities[0]*(reward+gamma*gw.state_values[new_state]) #probabilidad 0.8
                        
                    # si acción real es diferente a la acción que toma:  probabilidad 0.1
                    else: 
                        new_state, reward, _, done = gw.step(state, real_action, random=False)
                        suma_actions+=gw.action_probabilities[1]*(reward+gamma*gw.state_values[new_state]) #probabilidad 0.1
                
                print("Suma valor de acciones para acción "+str(action)+' : '+str( suma_actions) )
                if suma_actions>val_estado_opt:
                    val_estado_opt = suma_actions
                    accion_opt=action
        
            # actualizar valor de estado con el valor óptimo de la acción
            new_values[state]= val_estado_opt
            # actualizar política con acción óptima para el estado
            gw.policy[state] = accion_opt
            
            
            print("Valor de estado óptimo: "+str(new_values[state]) )
            print("Acción óptima en estado: "+str(gw.policy[state]) )
            print("")
            
    # Copiar valores
    value = copy.deepcopy(new_values) 
    gw.state_values = copy.deepcopy(new_values)         
    print("Valores de estado óptimos: "+str(new_values) )
    print("Política óptima: "+str (gw.policy))
    
        
value_iteration(gw, 0.9)

#gw.get_allowed_actions(state): Permite conocer el espacio de acciones dado un estado.
#gw.real_actions[action]: Las posibles acciones (reales) que puede tomar el agente dada una acción.
#gw.action_probabilities[action]: La probabilidad de que el agente tomé las acciones reales.
#gw.step(state, action, random): Permite ejecutar un paso en el gridworld (MDP). Con este método se puede conocer el estado siguiente, la recompensa, la accion tomada y la bandera de un estado terminal. Nota: si el parámetro random es Falso, las acciones son determinisiticas (con probabilidad 1, toma la acción dada).
#gw.state_values[state]: Los valores del estado state.
#gw.policy[state]: La acción dada por la política para el estado state

Política aleatoria incial: {(0, 0): 'N', (0, 1): 'E', (0, 2): 'E', (0, 3): 'E', (1, 0): 'N', (1, 2): 'N', (1, 3): 'N', (2, 0): 'E', (2, 1): 'N', (2, 2): 'S', (2, 3): 'E'}
Estado actual: (0, 0)
Suma valor de acciones para acción N : 0.5894191140883287
Suma valor de acciones para acción S : 0.5327872992189849
Suma valor de acciones para acción E : 0.644969148704666
Suma valor de acciones para acción W : 0.5733931355903742
Valor de estado óptimo: 0.644969148704666
Acción óptima en estado: E

Estado actual: (0, 1)
Suma valor de acciones para acción N : 0.6702998792691713
Suma valor de acciones para acción S : 0.6702998792691713
Suma valor de acciones para acción E : 0.7443801453743135
Suma valor de acciones para acción W : 0.5983661157276152
Valor de estado óptimo: 0.7443801453743135
Acción óptima en estado: E

Estado actual: (0, 2)
Suma valor de acciones para acción N : 0.7673859324901228
Suma valor de acciones para acción S : 0.5687327151749094
Suma valor de acciones para acción E : 0.84

In [ ]:
# EDITABLE
def policy_evaluation(gw, gamma):        
    g_tmp = {}
    
    
    for state in gw.states:
        # TO DO: Actualice los valores de los estados con Policy Evaluation
        
        
        
        
        
        
            
        gw.state_values[state] =  gw.max_val(gw.state_q_values[state])

Ahora implemente la función *policy improvement* que permitirá ajustar la política $\pi(s)$ de acuerdo a la nueva estimación de los valores $Q(s,a)$

In [63]:
#EDITABLE
def policy_improvement(gw):
    # Bandera de que la política es estable
    policy_stable = True
    
    # iterar sobre estados
    for state in gw.states:
        # TO DO: Actualice la política
        # best_action es accion máxima de valor de par estado-accion, para el estado actual
        best_action = max(gw.state_q_values[state])
        
        # si acción con maximo valor de par estado-accion, para el estado actual es diferentes a lo que dice la política actual
        if best_action!=gw.policy[state]:
            # actualizar política con acción con maximo valor de par estado-accion
            gw.policy[state]=best_action
            policy_stable = False
            
    return policy_stable

Ahora resuelva el gridworld por *policy iteration* utilizando las funciones que usted implementó:
* Incialice los estados iniciales
* Establezca un $\gamma=0.99$, un horizonte de 15 iteraciones y el estado inicial en (2,0)

In [64]:
# EDITABLE

# crear MDP=GridWorld
# params: 3 filas, 4 columnas,  (1,1)=celda bloqueada, (1,3)=bomba, (0,3)=diamante, 0.0=valor de estados inicial
gw = GridWorld(3, 4, [(1,1)], [(1,3)], [(0,3)], 0.0)
print("El nuevo MDP: "+str(gw) )

# Inicializar valores de los estados
gw.state_values = gw.init_values()
#print("Valores de estados iniciales: "+str(gw.state_values) )

# Se ajusta el valor de los estados terminales
# definir valor de estados terminales
gw.state_values[(0,3)] = 1.0
gw.state_values[(1,3)] = -1.0  
print("Ajuste de valores iniciales: "+str(gw.state_values) )
# TO DO: Actualice los valores q para Value iteration
#q = dict.fromkeys(gw.get_allowed_actions((state)), 0.0)        
#q

# valores de par estado-accion, para cada estado inicializar en 0
state_q_values = gw.init_qvalues()
print("Valores par estado-acción iniciales: "+str(state_q_values) )
gamma=0.99 # tasa descuento
H=15 # horizonte de tiempo

init_state = gw.state_values
init_state


El nuevo MDP: <gridworld.GridWorld object at 0x00000189490249D0>
Ajuste de valores iniciales: {(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (0, 3): 1.0, (1, 0): 0.0, (1, 2): 0.0, (1, 3): -1.0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0, (2, 3): 0.0}
Valores par estado-acción iniciales: {(0, 0): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}, (0, 1): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}, (0, 2): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}, (0, 3): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}, (1, 0): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}, (1, 2): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}, (1, 3): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}, (2, 0): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}, (2, 1): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}, (2, 2): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}, (2, 3): {'N': 0.0, 'S': 0.0, 'E': 0.0, 'W': 0.0}}


{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 1.0,
 (1, 0): 0.0,
 (1, 2): 0.0,
 (1, 3): -1.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 0.0,
 (2, 3): 0.0}

**Visualización:**
1. Ejecute la siguiente celda y haga clic sobre la ventana del gridworld. 
2. Podrá ir viendo como se actualizan los valores de cada estado cada vez que presionas la tecla **Espacio**. 
3. Tambien puede presionar la tecla Q en la ventana del gridworld para ver los valores Q de cada par (s,a)
4. Si tiene un error en la función 'policy_evaluation' o en 'policy_improvement', despues de corregirlo es necesario volver a correr las 5 celdas anteriores para poder volver a lanzar la interfaz del gridworld.
5. Una vez completadas las iteraciones, aparecerá un mensaje que dice TESTING. Presiona la tecla **Enter** para ver la política
6. Para cerrar el gridworld presiona la tecla **Esc**.

In [65]:
gw.policy_evaluation = MethodType(policy_evaluation, gw)
gw.policy_improvement = MethodType(policy_improvement, gw)
gw.solve_policy_iteration(gamma=gamma, horizon=H, init_state=init_state)

Estado actual: (0, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 1)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (0, 3)
Recompensa estado terminal: 1.0
Estado actual: (1, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (1, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (1, 3)
Recompensa estado terminal: -1.0
Estado actual: (2, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de

Estado actual: (0, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 1)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (0, 3)
Recompensa estado terminal: 1.0
Estado actual: (1, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (1, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (1, 3)
Recompensa estado terminal: -1.0
Estado actual: (2, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de

Estado actual: (0, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 1)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (0, 3)
Recompensa estado terminal: 1.0
Estado actual: (1, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (1, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (1, 3)
Recompensa estado terminal: -1.0
Estado actual: (2, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de

Estado actual: (0, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 1)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (0, 3)
Recompensa estado terminal: 1.0
Estado actual: (1, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (1, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (1, 3)
Recompensa estado terminal: -1.0
Estado actual: (2, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de

Estado actual: (0, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 1)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (0, 3)
Recompensa estado terminal: 1.0
Estado actual: (1, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (1, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (1, 3)
Recompensa estado terminal: -1.0
Estado actual: (2, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de

Estado actual: (0, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 1)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (0, 3)
Recompensa estado terminal: 1.0
Estado actual: (1, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (1, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (1, 3)
Recompensa estado terminal: -1.0
Estado actual: (2, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de

Estado actual: (0, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 1)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (0, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (0, 3)
Recompensa estado terminal: 1.0
Estado actual: (1, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado actual: (1, 2)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de estado V(s): 0.0

Estado terminal: (1, 3)
Recompensa estado terminal: -1.0
Estado actual: (2, 0)
Q(s,a) en acción: N
0.0
Q(s,a) en acción: S
0.0
Q(s,a) en acción: E
0.0
Q(s,a) en acción: W
0.0
Nuevo valor de

Con *policy iteration*, los valores de los estados son: 

In [ ]:
# EDITABLE


**Resultado esperado:**
```
V(0, 0) = 0.951
V(0, 1) = 0.964
V(0, 2) = 0.977
V(0, 3) = 1.000
V(1, 0) = 0.939
V(1, 2) = 0.890
V(1, 3) = -1.000
V(2, 0) = 0.925
V(2, 1) = 0.913
V(2, 2) = 0.900
V(2, 3) = 0.790
```


La política aprendida es para maximizar la recompensa desde el estado inicial (2,0) es:

In [ ]:
# EDITABLE


**Resultado esperado:**
```
Acción a tomar en (0, 0): E
Acción a tomar en (0, 1): E
Acción a tomar en (0, 2): E
Acción a tomar en (0, 3): N
Acción a tomar en (1, 0): N
Acción a tomar en (1, 2): W
Acción a tomar en (1, 3): N
Acción a tomar en (2, 0): N
Acción a tomar en (2, 1): W
Acción a tomar en (2, 2): W
Acción a tomar en (2, 3): S
```

Con esto termina la implementación de los algoritmos de Policy y Value iteration, puede vovlerlos a ejecutar como desee y variar sus parámetros para ver los efectos que estos tienen.

# Serpientes y Escaleras

In [6]:
from types import MethodType
from gridworld2 import GridWorld2, pygame
import copy
from colorama import Fore
import random

# crear MDP=GridWorld2 (Serpientes y Escaleras)
# params: 1 filas, 100 columnas,  []=celda bloqueada, (1,3)=bomba, (0,3)=diamante, 0.0=valor de estados inicial

diamantes = [(0,79),(0,99)]
bombas=[(0,22),(0,36),(0,44),(0,66),(0,88)]

gw = GridWorld2(1, 100, [], bombas, diamantes, 0.0)

# Inicializar valores y política
gw.state_values = gw.init_values()
gw.state_q_values = gw.init_qvalues()


# definir valor de estados 
for state in gw.state_values:
    if state in diamantes:
        gw.state_values[state] = 1.0
    elif state in bombas:
        gw.state_values[state] = -1.0
    else:
        gw.state_values[state] = 0.0
        
# definir política siempre acción='N'
gw.policy = dict.fromkeys(gw.states, 'N') # cambiar política
gw.updateGrid()


def nueva_casilla_azul(state):
    gw.state_values[state] = 1.0
    gw.updateGrid()
    
def nueva_casilla_roja(state):
    gw.state_values[state] = -1.0
    gw.updateGrid()
    

gw.state_values
    


{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (0, 5): 0.0,
 (0, 6): 0.0,
 (0, 7): 0.0,
 (0, 8): 0.0,
 (0, 9): 0.0,
 (0, 10): 0.0,
 (0, 11): 0.0,
 (0, 12): 0.0,
 (0, 13): 0.0,
 (0, 14): 0.0,
 (0, 15): 0.0,
 (0, 16): 0.0,
 (0, 17): 0.0,
 (0, 18): 0.0,
 (0, 19): 0.0,
 (0, 20): 0.0,
 (0, 21): 0.0,
 (0, 22): -1.0,
 (0, 23): 0.0,
 (0, 24): 0.0,
 (0, 25): 0.0,
 (0, 26): 0.0,
 (0, 27): 0.0,
 (0, 28): 0.0,
 (0, 29): 0.0,
 (0, 30): 0.0,
 (0, 31): 0.0,
 (0, 32): 0.0,
 (0, 33): 0.0,
 (0, 34): 0.0,
 (0, 35): 0.0,
 (0, 36): -1.0,
 (0, 37): 0.0,
 (0, 38): 0.0,
 (0, 39): 0.0,
 (0, 40): 0.0,
 (0, 41): 0.0,
 (0, 42): 0.0,
 (0, 43): 0.0,
 (0, 44): -1.0,
 (0, 45): 0.0,
 (0, 46): 0.0,
 (0, 47): 0.0,
 (0, 48): 0.0,
 (0, 49): 0.0,
 (0, 50): 0.0,
 (0, 51): 0.0,
 (0, 52): 0.0,
 (0, 53): 0.0,
 (0, 54): 0.0,
 (0, 55): 0.0,
 (0, 56): 0.0,
 (0, 57): 0.0,
 (0, 58): 0.0,
 (0, 59): 0.0,
 (0, 60): 0.0,
 (0, 61): 0.0,
 (0, 62): 0.0,
 (0, 63): 0.0,
 (0, 64): 0.0,
 (0, 65): 0.0,
 (0, 66): -1.0,
 

# Función de Valor

Siguiendo una política $\pi$,  el valor de un estado $V(s)$ en el tiempo $t$ formalmente corresponde a **la suma descontada de las recompensas recibidas, si el agente tiene como estado inicial $s$ y se comporta óptimamente en adelante**. En palabras simples, indica cuánta utilidad  recibiría el agente si comenzara en $s$ y se comportara bien de ahí en adelante, 

$$V_{t}(s)=\sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma V_{t-1}(s'))$$

In [3]:
# función de valor de estados: actualizar la función de valor de estados v(s) dada una politica pi
# parametros: gw=MDP, gamma=tasa de descuento

def update_values(gw, gamma):
    
    # vectores para los valores de estado
    # valores de estado, para cada estado inicializar en 0 
    value = dict.fromkeys(gw.states , 0.0)
    # nuevos valores de estado, para cada estado inicializar en 0
    new_values = dict.fromkeys(gw.states , 0.0)
    
    
    # iterar sobre los estados posibles del MDP
    for state in gw.states: 
        # Tomar acción de politica dada para el estado
        action = gw.policy[state]
        
        # Dar un paso en par estado-accion (random=False deterministico)
        # retorna: estado siguiente, recompensa, accion
        new_state, reward, _, done = gw.step(state, action, random=False)

        # Actualizar valores de estado
        if(done): # llega a estado terminal
            # actualización para un estado terminal
            new_values[state] = reward # valor de estado v(s) para estado terminal es recompensa
        else:
            
            # valor de estado v(s) es la suma descontada de recompensas de las acciones posibles s'
            suma_actions = 0.0
            
            # iterar sobre Las posibles acciones (reales) que puede tomar el agente dada una acción
            for real_action in  gw.real_actions[action]:
                # si acción real es igual a la acción que toma: probabilidad 0.8
                if real_action==action: 
                    new_state, reward, _, done = gw.step(state, real_action, random=False)
                    suma_actions+=gw.action_probabilities[0]*(reward+gamma*gw.state_values[new_state]) #probabilidad 0.8
                        
                # si acción real es diferente a la acción que toma:  probabilidad 0.2
                else: 
                    new_state, reward, _, done = gw.step(state, real_action, random=False)
                    suma_actions+=gw.action_probabilities[1]*(reward+gamma*gw.state_values[new_state]) #probabilidad 0.2
                    
            print("Suma valor de acciones para acción "+str(action)+' : '+str( suma_actions) )
            # actualización para un estado no terminal
            new_values[state] = suma_actions
                
            
    # Copiar valores
    value = copy.deepcopy(new_values) 
    gw.state_values = copy.deepcopy(new_values) 
    

In [4]:
gamma=0.9
H=15
gw.update_values = MethodType(update_values, gw)
gw.solve_dynamic_programming(gamma=gamma, horizon=H)

NameError: name 'MethodType' is not defined

# Value iteration

In [ ]:
# EDITABLE

# value_iteration: encontrar valores de estado óptimos para cada estado y politica óptima(acción óptima) para cada estado 
# params: gw=MDP gridworld, gamma=descuento

def value_iteration(gw,gamma):   
    
    # valores de estado, para cada estado inicializar en 0 
    value = dict.fromkeys(gw.states , 0.0)
    
    # nuevos valores de estado, para cada estado inicializar en 0
    new_values = dict.fromkeys(gw.states , 0.0)    
    
    
    # politica aleatoria 
    # inicializar politica aleatoria 
    gw.policy = {}
    keys = gw.states # keys del diccionario son los estados posibles del MDP
    
    # iterar sobre estados posibles del MDP
    for state in keys:    
        # inicializar valores de política aleatoria escogiendo acción aleatoria en estado 
        gw.policy[state] = random.choice(gw.get_allowed_actions(state) )
    print("Política aleatoria incial: "+str(gw.policy) )    
    
    # iterar sobre estados
    for state in gw.states:          
        
        # Dar un paso en par estado-accion con accion dada por la política aleatoria
        new_state, reward, _, done = gw.step(state, gw.policy[(state)], random=True)
        
        if(done==True): # llega a estado terminal
            # actualización para un estado terminal        
            new_values[state] = reward # valor de estado para estado terminal es recompensa
            print("Estado terminal: "+str(state) )
            print("Recompensa estado terminal: "+str(reward) )
        else: # llega a estado no terminal
            # actualización para un estado no terminal
            
            accion_opt = -1 # acción óptima en estado (política)
            val_estado_opt = -100 # valor óptimo en estado 
            print("Estado actual: "+str(state) )
            # iterar sobre acciones permitidas en estado state
            for action in gw.get_allowed_actions(state):
                
                suma_actions = 0 # suma de valores de par estado-accion real que puede tomar (en un estado)
                # iterar sobre Las posibles acciones (reales) que puede tomar el agente dada una acción
                for real_action in  gw.real_actions[action]:
                    # si acción real es igual a la acción que toma: probabilidad 0.8
                    if real_action==action: 
                        new_state, reward, _, done = gw.step(state, real_action, random=False)
                        suma_actions+=gw.action_probabilities[0]*(reward+gamma*gw.state_values[new_state]) #probabilidad 0.8
                        
                    # si acción real es diferente a la acción que toma:  probabilidad 0.1
                    else: 
                        new_state, reward, _, done = gw.step(state, real_action, random=False)
                        suma_actions+=gw.action_probabilities[1]*(reward+gamma*gw.state_values[new_state]) #probabilidad 0.1
                
                print("Suma valor de acciones para acción "+str(action)+' : '+str( suma_actions) )
                if suma_actions>val_estado_opt:
                    val_estado_opt = suma_actions
                    accion_opt=action
        
            # actualizar valor de estado con el valor óptimo de la acción
            new_values[state]= val_estado_opt
            # actualizar política con acción óptima para el estado
            gw.policy[state] = accion_opt
            
            
            print("Valor de estado óptimo: "+str(new_values[state]) )
            print("Acción óptima en estado: "+str(gw.policy[state]) )
            print("")
            
    # Copiar valores
    value = copy.deepcopy(new_values) 
    gw.state_values = copy.deepcopy(new_values)         
    print("Valores de estado óptimos: "+str(new_values) )
    print("Política óptima: "+str (gw.policy))
        
value_iteration(gw, 0.9)

In [ ]:
gamma=0.9
H=15
gw.value_iteration = MethodType(value_iteration, gw)
gw.solve_value_iteration(gamma=gamma, horizon=H, init_state=init_state )